In [1]:
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import re
from nltk.corpus import stopwords
import nltk
import string
import matplotlib.pyplot as plt
import random
%matplotlib inline

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn import decomposition, ensemble
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
#from nltk.corpus import stopwords
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier

from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix

import joblib


### Part 1  prepare for training data 

In [2]:
ibc_df = pd.read_csv('/home/jovyan/work/Model_Training_Data/IBC_data_260319_1.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# remove the 4 specific deletion reasons from dataset
listreason=['incorrect_year_of_service','Incorrect Specification - Non-Risk Adjusted','repeated_instance', 'incorrect_spec']
ibc_df=ibc_df[ibc_df['deleted_reason'].isin(listreason)== False]
ibc_df['deleted_reason'].unique()

array([nan, 'not_doctors_note', 'phoneORaddressORother_number',
       'not_relevant', 'preventative_screening', 'negation', 'other',
       'family_history', 'incorrect_acronym', 'medical_history',
       'incorrect_version'], dtype=object)

In [4]:
ibc_df.head()

,Unnamed: 0,chart_id,code_id,text,start_offset,end_offset,deleted_reason,deleted,label,15char,25char,50char,75char,10word_after,10word_before,50word_after,50word_before,flag
0,0,002803945_161667176A_2016_001,E6601,e66.01,14251,14257,NaN,NaN,agreed,cess calories (E66.01).,due to excess calories (E66.01). ...,sessment Morbid obesity due to excess calori...,.7). Assessment Morbid obe...,"[[').', 'Assessment', 'Traumatic', 'brain', 'i...","[['Fibromyalgia', '(M79.7).', 'Assessment', 'M...","[[').', 'Assessment', 'Traumatic', 'brain', 'i...","[['Assessment/Plan', '#', 'Detail', 'Type', 'D...",0
1,1,002803945_161667176A_2016_001,E6601,e66.01,22445,22451,NaN,NaN,agreed,cess calories (E66.01). Impre,due to excess calories (E66.01). Impre...,sessment Morbid obesity due to excess calori...,ise. Assessment Morbid obe...,"[[').', 'Impression', 'encouraged', 'to', 'die...","[['-diet', 'exercise.', 'Assessment', 'Morbid'...","[[').', 'Impression', 'encouraged', 'to', 'die...","[['Impression', 'seeing', 'pain', 'management,...",0
2,2,002803945_161667176A_2016_001,E789,e78.9,14501,14506,NaN,NaN,agreed,m cholesterol (E78.9).,vated serum cholesterol (E78.9). ...,Assessment Elevated serum cholester...,izures (F44.5). Assessment ...,"[[').', 'Assessment', 'Anxiety', '(F41.9).', '...","[['sequela', '(S06.9X9S).', 'Assessment', 'Pse...","[[').', 'Assessment', 'Anxiety', '(F41.9).', '...","[['chronic', 'depression--multitude', 'sx', 's...",0
3,3,002803945_161667176A_2016_001,F329,f32.9,21557,21562,NaN,NaN,agreed,"sion, chronic (F32.9). Impression","jor depression, chronic (F32.9). Impressions...","ad injury. AssessmentMajor depression, chron...","etter w meds, flared w head injury. Assessme...","[[').', 'Impressionstable,', 'cont', 'Rx.', 'A...","[['w', 'meds,', 'flared', 'w', 'head', 'injury...","[[').', 'Impressionstable,', 'cont', 'Rx.', 'A...","[['accupuncturist,pain', 'management.', 'Asses...",0
4,4,002803945_161667176A_2016_001,F329,Major depression,17574,17590,NaN,NaN,agreed,"pap finding Major depression,08/10/2016 ch","abnormal pap finding Major depression,08/1...",:00 PM 3/36 unspecified abnormal pap findin...,07/28/1977 09/19/2016 06:00 PM 3/36 unspeci...,"[[',08/10/2016', 'chronic', 'Left-sided', 'wea...","[['I.', '07/28/1977', '09/19/2016', '06:00', '...","[[',08/10/2016', 'chronic', 'Left-sided', 'wea...","[['on', '05/04/2014.', 'Hallucination', '08/10...",0


In [5]:
# doing some preprocessing or cleaning punctuation and stop words
ibc_df= ibc_df.replace(np.nan,'',regex = True)
ibc_df=ibc_df
ibc_df['text75'] = ibc_df['75char']
ibc_df = ibc_df[pd.notnull(ibc_df['text75'])] 
ibc_df['cleaned_data'] = ibc_df['text75'].apply(lambda x: x.lower())
ibc_df['cleaned_data'] = ibc_df['cleaned_data'].apply(lambda x: x.translate(str.maketrans('','',string.punctuation)))

# remove stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

ibc_df['cleaned_data']= ibc_df['cleaned_data'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
ibc_df['cleaned_data']= ibc_df['cleaned_data'].str.findall('\w{2,}').str.join(' ')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
ibc_df.head()

,Unnamed: 0,chart_id,code_id,text,start_offset,end_offset,deleted_reason,deleted,label,15char,25char,50char,75char,10word_after,10word_before,50word_after,50word_before,flag,text75,cleaned_data
0,0,002803945_161667176A_2016_001,E6601,e66.01,14251,14257,,,agreed,cess calories (E66.01).,due to excess calories (E66.01). ...,sessment Morbid obesity due to excess calori...,.7). Assessment Morbid obe...,"[[').', 'Assessment', 'Traumatic', 'brain', 'i...","[['Fibromyalgia', '(M79.7).', 'Assessment', 'M...","[[').', 'Assessment', 'Traumatic', 'brain', 'i...","[['Assessment/Plan', '#', 'Detail', 'Type', 'D...",0,.7). Assessment Morbid obe...,assessment morbid obesity due excess calories ...
1,1,002803945_161667176A_2016_001,E6601,e66.01,22445,22451,,,agreed,cess calories (E66.01). Impre,due to excess calories (E66.01). Impre...,sessment Morbid obesity due to excess calori...,ise. Assessment Morbid obe...,"[[').', 'Impression', 'encouraged', 'to', 'die...","[['-diet', 'exercise.', 'Assessment', 'Morbid'...","[[').', 'Impression', 'encouraged', 'to', 'die...","[['Impression', 'seeing', 'pain', 'management,...",0,ise. Assessment Morbid obe...,ise assessment morbid obesity due excess calor...
2,2,002803945_161667176A_2016_001,E789,e78.9,14501,14506,,,agreed,m cholesterol (E78.9).,vated serum cholesterol (E78.9). ...,Assessment Elevated serum cholester...,izures (F44.5). Assessment ...,"[[').', 'Assessment', 'Anxiety', '(F41.9).', '...","[['sequela', '(S06.9X9S).', 'Assessment', 'Pse...","[[').', 'Assessment', 'Anxiety', '(F41.9).', '...","[['chronic', 'depression--multitude', 'sx', 's...",0,izures (F44.5). Assessment ...,izures f445 assessment elevated serum choleste...
3,3,002803945_161667176A_2016_001,F329,f32.9,21557,21562,,,agreed,"sion, chronic (F32.9). Impression","jor depression, chronic (F32.9). Impressions...","ad injury. AssessmentMajor depression, chron...","etter w meds, flared w head injury. Assessme...","[[').', 'Impressionstable,', 'cont', 'Rx.', 'A...","[['w', 'meds,', 'flared', 'w', 'head', 'injury...","[[').', 'Impressionstable,', 'cont', 'Rx.', 'A...","[['accupuncturist,pain', 'management.', 'Asses...",0,"etter w meds, flared w head injury. Assessme...",etter meds flared head injury assessmentmajor ...
4,4,002803945_161667176A_2016_001,F329,Major depression,17574,17590,,,agreed,"pap finding Major depression,08/10/2016 ch","abnormal pap finding Major depression,08/1...",:00 PM 3/36 unspecified abnormal pap findin...,07/28/1977 09/19/2016 06:00 PM 3/36 unspeci...,"[[',08/10/2016', 'chronic', 'Left-sided', 'wea...","[['I.', '07/28/1977', '09/19/2016', '06:00', '...","[[',08/10/2016', 'chronic', 'Left-sided', 'wea...","[['on', '05/04/2014.', 'Hallucination', '08/10...",0,07/28/1977 09/19/2016 06:00 PM 3/36 unspeci...,07281977 09192016 0600 pm 336 unspecified abno...


In [7]:
save4testibclist=pd.read_csv('save4testibclistlstm.csv')
testlist2000=save4testibclist.columns

In [8]:
train_df = ibc_df[ibc_df['chart_id'].isin(testlist2000)== False]
test_df = ibc_df[ibc_df['chart_id'].isin(save4testibclist)]

In [9]:
print(len(train_df))
print(len(test_df))

349324
18127


In [10]:
#train_df=train_df[['chart_id','code_id','text','deleted_reason','deleted','label','flag','cleaned_data']]
#test_df=test_df[['chart_id','code_id','text','deleted_reason','deleted','label','flag','cleaned_data']]

In [11]:
train_df.to_csv('ibc_trainingdata_040219.csv')
test_df.to_csv('ibc_testingdata_040219.csv')